In [4]:
import speech_recognition as sr
import os
import argparse
import subprocess
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn import neighbors
import numpy as np
import time
import wave
import pyaudio
import sys

In [63]:
#for indian english
r = sr.Recognizer()
with sr.Microphone() as source:
    print("start speaking")
    r.adjust_for_ambient_noise(source, duration = 3)
    audio_data = r.record(source,duration=20)
try:
    text = r.recognize_google(audio_data,language="en-IN")
    print("you said:{}".format(text))
except Exception as e:
    print(e)
with open("voice.wav", "wb") as file:
    file.write(audio_data.get_wav_data())


start speaking
you said:together to form a group that is usually longer than a sentence paragraph of Aamir up of several sentences in most organised forms of writing such as acids paragraph container topic sentence all these years you look too in many instances we don't really define what the idea behind the paragraph is this is one of those


In [49]:
#for indian hindi
with sr.Microphone() as source:
    print("start speaking")
    r.adjust_for_ambient_noise(source, duration = 3)
    audio_data = r.record(source,duration=20)
try:
    text = r.recognize_google(audio_data,language="hi-IN")
    print("you said:{}".format(text))
except Exception as e:
    print(e)


start speaking
you said:मैं यह बना रहा हूं और तुम कैसे हो शुक्रिया


In [ ]:
#for french
with sr.Microphone() as source:
    print("start speaking")
    r.adjust_for_ambient_noise(source, duration = 3)
    audio_data = r.record(source,duration=20)
try:
    text = r.recognize_google(audio_data,language="hi-IN")
    print("you said:{}".format(text))
except Exception as e:
    print(e)


In [52]:
#import dataset for making the model for gender recognition by voice
def import_dataset(dataset_path):
    
    df_voice=pd.read_csv(dataset_path)
    df_voice = shuffle(df_voice)
    
    #replacing the labels male->0 and female ->1
    df_voice["label"] = df_voice["label"].replace("male",0)
    df_voice["label"] = df_voice["label"].replace("female",1)
    
    
    
    X=df_voice.iloc[:,:-1].values
    Y=df_voice.iloc[:,-1].values
    
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=1/3,random_state=0)
    
    return X_train,Y_train,X_test,Y_test

In [53]:
x_train, y_train, x_test, y_test = import_dataset("dataset/voice.csv")
#normalizing the dataset
norm = Normalizer(norm='l2')
x_train = norm.transform(x_train)
x_test = norm.transform(x_test)


In [54]:
#applying pca decompostion
pca = PCA()
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)

In [55]:
def predict_and_score(lr,x_test,y_test):
    p_test=lr.predict(x_test)
    return accuracy_score(y_test,p_test)

In [56]:
#logistic regression
lr=LogisticRegression()
lr.fit(x_train,y_train.ravel())#numpy.ravel() is used to flatten the array
acc = predict_and_score(lr,x_test,y_test)
print("LR: ",acc)

#bernoulli naive Bayes
nb = BernoulliNB()
nb.fit(x_train,y_train.ravel())
acc = predict_and_score(nb,x_test,y_test)
print("NB: ",acc)

#svm gamma = "auto"
clf = svm.NuSVC(gamma="auto")
clf.fit(x_train,y_train.ravel())
acc = predict_and_score(clf,x_test,y_test)
print("svm: ",acc)

#knn 
knn = neighbors.KNeighborsClassifier(11,algorithm="ball_tree",weights="distance")
knn.fit(x_train,y_train.ravel())
acc = predict_and_score(clf,x_test,y_test)
print("knn: ",acc)

E:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR:  0.665719696969697
NB:  0.8589015151515151
svm:  0.9119318181818182
knn:  0.9119318181818182


In [64]:
p1 = subprocess.run([r"C:\Program Files\R\R-3.6.1\bin\x64\Rscript.exe",r"C:\Users\This pc\extract_single.r"],capture_output=True,text=True)
print(p1.stderr)

Warning message:
package 'tuneR' was built under R version 3.6.3 
Warning message:
package 'seewave' was built under R version 3.6.3 
Loading required package: NatureSounds
Loading required package: knitr

NOTE: functions are being renamed (run 'print(new_function_names)' to see new names). Both old and new names are available in this version 
 Please see citation('warbleR') for use in publication
Warning messages:
1: package 'warbleR' was built under R version 3.6.3 
2: package 'NatureSounds' was built under R version 3.6.3 
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.2.0     v purrr   0.3.2
v tibble  2.1.3     v dplyr   0.8.3
v tidyr   0.8.3     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
x readr::spec()   masks seewave::spec()
Warning message:
In (0:(n - 1)) * f : NAs pro

In [65]:
new_df = pd.read_csv("voiceDetails.csv")
new_df = new_df.drop(axis=1,columns=["sound.files","selec","duration","peakf"])

In [66]:
new_sample =new_df.values
print(new_sample.shape)

(1, 20)


In [67]:
new_sample =norm.transform(np.float64(new_sample))

In [68]:
new_sample=pca.transform(new_sample)[0]

In [69]:
print(new_sample)
if clf.predict([new_sample])==1 :
    print("\n Female")
else:
    print("\n Male")


[ 3.44770952e-01  2.49094564e-02 -4.52248592e-02 -8.52970632e-03
 -4.46182364e-02  3.79467206e-03  8.34416455e-03  4.35847788e-03
  1.74047291e-03  1.90446245e-03  4.58510561e-05  1.17986646e-03
 -1.16758694e-03  4.35059088e-04  4.73593724e-04  4.42824205e-04
 -1.28502291e-05  4.94210465e-16  1.50566072e-17 -5.69985038e-20]

 Female
